In [ ]:
import tensorflow as tf

In [ ]:
#一般的写法，即不做梯度截断时的正常优化
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3) #定义优化器，设定学习率
grads_and_vars=optimizer.compute_gradients(loss) #对loss求梯度
tr_op_set = optimizer.apply_gradients(grads_and_vars, global_step=global_step) #更新参数

#因为也没有做梯度截断操作，故而上面三句也可以直接写成一句话
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(loss)

In [ ]:
#做梯度截断时候的方式：
#方式一：tf.clip_by_value  #纯数值截断，即自定义上下界，即同时防止爆炸和消失
#方式二：tf.clip_by_global_norm  #这个严格来说不是截断，是按一定比例变小，主要是为了防止梯度爆炸
#方式三：tf.clip_by_norm  #这个和第二种方式一个性质，也是让梯度变小，和上面的区别是固定地除上梯度g的l2正则

#对于计算梯度
#则即可以是opt.compute_gradients(loss,params) #默认params都是trainable的变量
#也可以是tf.gradients(loss, params) 

In [ ]:
#方式一：
grads_vars = self.opt.compute_gradients(self.loss) #根据损失，计算梯度，默认也是针对trainable_variables
#返回的是 A list of (gradient, variable) pairs.故而grads_vars是一个列表
capped_grads_vars = [[tf.clip_by_value(g, -self.config["clip"], self.config["clip"]), v]
                     for g, v in grads_vars] #截断梯度，前后都截断，防止梯度爆炸和消失，纯数值截断，比如大于5的梯度都设置为5，比-5小的设置为-5
#这里选择的是前后都截断，也可以选择下面的方式，即按一定比例变小
self.train_op = self.opt.apply_gradients(capped_grads_vars, self.global_step) #更新参数
#注意，这里需要把train_op放sess.run里面，才会去更新

In [ ]:
#方式二：
params = tf.trainable_variables()
gradients = tf.gradients(loss, params) 
self.updates = []
#通过损失对需要更新的参数进行求梯度，更新选择的参数。
#tf.gradients最重要的即两个参数，一个即ys，这里即loss，另一个即xs，这里即要更新的params，即用ys对xs进行求偏导
clipped_gradients, norm = tf.clip_by_global_norm( gradients, clip_norm) #t_list[i] * clip_norm / max(global_norm, clip_norm)
                                                                        #global_norm = sqrt(sum([l2norm(t)**2 for t in t_list]))
self.gradient_norms.append(norm)
self.updates.append(opt.apply_gradients(zip(clipped_gradients, params))) #反向传播，更新参数
#同样的，这里的updates也要放在sess.run里面，即只有updates被运行了，参数才会被更新
#当然，最后一句话换成以下的也可以，同样是为了将梯度更新到参数，这里因为是多组，所以用一个列表接收
#self.train_op = opt.apply_gradients(zip(clipped_gradients, params))

In [ ]:
#方式三：
optimizer = tf.train.AdamOptimizer(learning_rate, beta1=0.5)
grads = optimizer.compute_gradients(loss)
for i, (g, v) in enumerate(grads):
    if g is not None:
        grads[i] = (tf.clip_by_norm(g, 5), v)  # 即对每个g做以下处理：`g * clip_norm / l2norm(g)`,相比global_norm少了一个max
train_op = optimizer.apply_gradients(grads)